# Ch7. 유동성 모델링

- 유동성 위험의 원인: 완전 시장과 대칭적 정보 패러다임으로부터의 이탈 -> 도덕적 해이와 역선택으로 이어질 수 있음
- 유동성을 고려한 모델: 유동성이 요구되는 자산 수익과 불확실성 수준 모두에 영향을 미침 ~ more applicable
- 유동성: 채무불이행 가능성 추정에 있어 상당히 중요함
- 유동성 자산: 상당한 가격 영향 없이 대량의 자산이 판매될 수 있는 상태 ~ 거래 비용이라고도 함

(\*) 유동성을 정의하는 네 가지 특성
- 집중도: 자산을 같은 가격에 동시에 거래할 수 있는 능력 ~ 거래 비용 높으면 매수와 매도 가격의 차이 큼
- 즉시성: 대량의 매수 or 매도 주문을 거래할 수 있는 속도
- 깊이: 다양한 가격으로 풍부한 주문을 처리할 수 있는 많은 수의 구매자와 판매자가 존재하는지
- 탄력성: 불균형에서 회복하는 시장의 능력 ~ 주문 불균형이 빠르게 해소되는 가격 회복 과정

- 군집화 분석 -> 유동성 측정 군집화 -> 투자자가 유동성의 어느 부분에 집중해야 하는지 이해
- 군집화 분석: by 가우스 혼합 모델(GMM), 가우스 혼합 코풀라 모델(GMCM)
- GMM: 타원 분포에 잘 작동
- GMCM: 상관관계 고려 ~ GMM의 확장

(\*) 유동성 척도: 거래량 / 거래 비용 / 가격 영향 / 시장 영향

1. 거래량 기반 유동성 척도
- 시장이 깊을 때, 즉 금융 시장이 풍부한 주문을 충족할 수 있는 능력이 있을 때 대규모 주문이 처리됨
- 시장에 깊이가 없으면, 주문 불균형과 불연속성이 나타남

(i) 유동성 비율: 1%의 가격 변동을 유발하는 데 필요한 거래량의 정도 측정
$\\ LR_{it}=\frac{\sum^T_{t=1}P_{it}V_{it}}{\sum^T_{t=1}|PC_{it}}$
- $P_{it}$: t일의 주식 i의 총 가격
- $V_{it}$: t일의 주식 i의 거래량
- $|PC_{it}|$: t일과 t-1 에서의 가격 차이의 절댓값
- 비율이 높을수록 자산 i의 유동성이 높아짐 ~ 높은 거래량 / 낮은 가격차

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize']=(10,6)
pd.set_option('use_inf_as_na', True)

: 